# Creating a new dataset

In [39]:
from bs4 import BeautifulSoup as bs
import requests
import smtplib
import time
import datetime

# https://python-forum.io/thread-38862.html

In [40]:
# Connect to website
url = 'https://www.amazon.com/Momentive-Performance-Materials-GE284-Waterproof/dp/B001JK5R0I/ref=pd_sim_cpf_related_desktop_d_sccl_3_1/133-8736340-3018253?pd_rd_i=B001JK5R0I&psc=1'

# choosing the right kind of header helped me a lot in this
headers = {
    'authority': 'www.amazon.com',
    'pragma': 'no-cache',
    'cache-control': 'no-cache',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

page = requests.get(url, headers=headers)

# Pulling in the content from the page
soup1 = bs(page.text,'lxml')
soup2 = bs(soup1.prettify(),'lxml')

title = soup2.find(id='productTitle').get_text()
price = soup2.find(class_='a-offscreen').get_text()

print(title)
print(price)


           Momentive Performance Materials GE284 2.8-Ounce Silicone II Premium Waterproof Kitchen and Bath Squeeze Tube, Clear
          

                         $8.99
                        


In [41]:
# cleaning the data from either side of the data that we got from website
# since we want the price data to have only numeric values and no dollar sign, 
# we will strip the dollar sign off that as well

price = price.strip()[1:]
title = title.strip()
# print(title)
print(price)

8.99


In [42]:
type(price)

str


### Inserting/ writing the data gathered from the website into a newly created csv file by us:
#### This is for initial insertion of data where we define the column/ headers of the file the appending of the data in this very csv file is different. We will also be adding date of the data imported, its never not useful.

In [48]:
import datetime
today = datetime.date.today()
print(today)

2024-03-01


In [49]:
import csv
header = ['Title', 'Price', 'Date']

# changing the data type of title and price to list
data = [title, price, today]

type(data)

list

In [50]:
# can comment this out later in order to not delete your stored data
with open('AmazonWebScraper.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [56]:
import pandas as pd
df = pd.read_csv(r'E:\Data Science\SQL\Projects\3 Amazon Web Scraping Using Python\AmazonWebScraper.csv')
df

,Title,Price,Date
0,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
1,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
2,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
3,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
4,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01


## Appending data to the csv file now:

#### Here, we wont be required to add the header anymore since we already did that initially, and also, changing the 'w' to '+a' indicating that we want to append and not write data now.

In [57]:
with open('AmazonWebScraper.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

### We want that the data is appended on its own without us doing that manually everyday, when the data increases. We will set a timer.

In [64]:
def send_mail():
    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    server.echlo()
    server.echlo()
    server.login("aryashweta46@gmail.com", "")
    
    subject = 'The product is below this now'
    body = "Just buy it!"
    
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'aryashweta46@gmail.com', msg
    )

In [63]:


def check_price():
    url = 'https://www.amazon.com/Momentive-Performance-Materials-GE284-Waterproof/dp/B001JK5R0I/ref=pd_sim_cpf_related_desktop_d_sccl_3_1/133-8736340-3018253?pd_rd_i=B001JK5R0I&psc=1'
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    
    page = requests.get(url, headers=headers)
    
    # Pulling in the content from the page
    soup1 = bs(page.text,'lxml')
    soup2 = bs(soup1.prettify(),'lxml')
    
    title = soup2.find(id='productTitle').get_text()
    price = soup2.find(class_='a-offscreen').get_text()
    
    price = price.strip()[1:]
    title = title.strip()
    
    import datetime
    today = datetime.date.today()
    
    import csv
    header = ['Title', 'Price', 'Date']
    data = [title, price, today]
    
    with open('AmazonWebScraper.csv', '+a', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

    # sending yourself a mail in case the price of the product drops below a certain level
    if(price < 7):
        send_mail()

In [ ]:
# setting the timer

while(True):
    check_price()
    time.sleep(86400)    
# you can out in the time inside the sleep in seconds for which you want the loop to repeat itself

In [61]:
import pandas as pd
df = pd.read_csv(r'E:\Data Science\SQL\Projects\3 Amazon Web Scraping Using Python\AmazonWebScraper.csv')
df

,Title,Price,Date
0,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
1,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
2,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
3,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
4,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
5,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
6,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
7,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
8,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
9,Momentive Performance Materials GE284 2.8-Ounc...,8.99,2024-03-01
